In [ ]:
import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer
import numpy as np
import pickle
import json
import os
from collections import Counter
from sklearn.metrics import mean_squared_error
from typing import Dict, List
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
import math
#define utility functions

def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    counts = result.get_counts()
    return counts
'''
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram
'''

def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive


def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """Returns the number of gate operations with each number of qubits."""
    counter = Counter([len(gate[1]) for gate in circuit.data])
    #feel free to comment out the following two lines. But make sure you don't have k-qubit gates in your circuit
    #for k>2
    #for i in range(2,20):
     #   assert counter[i]==0
        
    return counter


def image_mse(image1,image2):
    # Using sklearns mean squared error:
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    return mean_squared_error(255*image1,255*np.array(image2))

In [ ]:
data_path='data'
#load the actual hackthon data (fashion-mnist)
images=np.load(data_path+'/images.npy')
labels=np.load(data_path+'/labels.npy')
#you can visualize it
import matplotlib.pyplot as plt
plt.imshow(images[1])

In [ ]:
#submission to part 1, you should make this into a .py file

n=len(images)
mse=0
gatecount=0

# Functions 'encode' and 'decode' are dummy.
def encode(image):
    p1 = []
    a, b = len(image), len(image[0])
    for in_1 in image:
        for in_2 in in_1:
            p1.append(in_2)
    n = bin(a * b)[2:]
    if a * b == 2 ** (len(n) - 1):
        n = n[:-1]
    x = (2 ** len(n)) - len(p1)
    for count in range(x):
        p1.append(0)
    qc = QuantumCircuit(len(n) + 1)
    lar = 0
    for e in p1:
        lar += e ** 2
    a = 1 / math.sqrt(lar)
    for count in range(len(p1)):
        p1[count] = a * p1[count]
    qc.cx(0,1)
    qc.ccx(0,2,1)
    qc.initialize(p1, qc.qubits[:-1])
    q = []
    if lar <= 1:
        q = [math.sqrt(lar / 2.0), math.sqrt(1 - (lar / 2.0))]
    else:
        q = [math.sqrt(0.5 + (1 / (2 * lar))), math.sqrt(1 - (0.5 + (1 / (2 * lar))))]
    qc.initialize(q, qc.qubits[-1])
    qc.measure_all()
    # qc.save_statevector()
    return qc

def decode(hist_i):
    s = 28
    kr = list(hist_i.keys())
    hist = {}
    scaler = [0,0]
    for elemen in hist_i:
        if elemen[1:] not in list(hist.keys()):
            hist[elemen[1:]]=0
        hist[elemen[1:]] += hist_i[elemen]
        scaler[int(elemen[0])] += hist_i[elemen]
    c = list(hist.keys())
    if scaler[0] > scaler[1]:
        lar = 1/(((scaler[0]/sum(scaler))-0.5)*2.0)
    else:
        lar = 2*(scaler[0]/sum(scaler))
    a = c[0]
    b = len(a)
    p = [0 for x in range(2**b)]
    for count in range(len(p)):
        if bin(count)[2:] in c:
            p[count] = hist[bin(count)[2:]]
        else:
            p[count] = 0
    saurkraut = sum(p)
    alarm_state = 0
    if saurkraut ==0:
        for count in range(len(p)):
            p[count] = 0
    else:
        for count in range(len(p)):
            p[count] = lar * math.sqrt(p[count]*(1/saurkraut))
            if p[count] > 1.0:
                alarm_state = max(alarm_state,p[count])
    if alarm_state != 0:
            skill_issue = 1.0/alarm_state
    else:
        skill_issue = 1.0
    for rizz in range(len(p)):
        p[rizz] = skill_issue*p[rizz]
    ans = []
    temp = []
    for count in range(len(p)):
        if len(ans) == s:
            break
        temp.append(p[count])
        if len(temp) == s:
            ans.append(temp)
            temp = []
    return ans

def run_part1(image):
    #encode image into a circuit
    circuit=encode(image)

    #simulate circuit
    histogram=simulate(circuit)

    #reconstruct the image
    image_re=decode(histogram)
    return circuit,image_re

In [ ]:
#how we grade your submission

n=len(images)
mse=0
gatecount=0

for data in images:
    #encode image into circuit
    circuit,image_re=run_part1(data)
    
    #count the number of 2qubit gates used
    gatecount+=count_gates(circuit)[2]
    
    #calculate mse
    mse+=image_mse(data,image_re)
    
#fidelity of reconstruction
f=1-mse/n
gatecount=gatecount/n

#score for part1 
print(f*(0.999**gatecount))